# Exploration notebook

* [Imports](#imports)
* [Data loading](#data-loading)
* [Missing values](#missing-values)
    * [Quantification](#quantification)
    * [Imputation](#imputation)
* [Data filtering](#data-filtering)
    * [Row filter](#row-filter)
    * [Column filter](#column-filter)
* [Distributions](#distributions)
    * [Numerical features](#numerical-features)
    * [Categorical features](#categorical-features)
* [Correlations](#correlations)
* [Feature engineering](#feature-engineering)
    * [Feature engineering](#feature-engineering)
* [Exports](#exports)


<a name="imports"></a>
## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import missingno as msno

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from scipy import stats

<a name="data-loading"></a>
## Data loading

In [ ]:
file_name_2016 = "data_2016.csv"
if not os.path.exists(file_name_2016):
    !wget https://data.seattle.gov/api/views/2bpz-gwpy/rows.csv?accessType=DOWNLOAD -q --show-progress -O $file_name_2016
!head -2 $file_name_2016

In [ ]:
file_name_2015 = "data_2015.csv"
if not os.path.exists(file_name_2015):
    !wget https://data.seattle.gov/api/views/h7rm-fz6m/rows.csv?accessType=DOWNLOAD -q --show-progress -O $file_name_2015
!head -2 $file_name_2015

In [ ]:
df_2015 = pd.read_csv(file_name_2015)
df_2016 = pd.read_csv(file_name_2016)
print(df_2016.columns)
print([col for col in df_2015.columns if col not in df_2016.columns])
print([col for col in df_2016.columns if col not in df_2015.columns])
df_2016.head()

In [ ]:
df_2015 = df_2015.rename(columns={'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions', 'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity'})

In [ ]:
print([col for col in df_2015.columns if col not in df_2016.columns])
print([col for col in df_2016.columns if col not in df_2015.columns])

In [ ]:
df_2015['TotalGHGEmissions'].describe()

In [ ]:
df_2016['TotalGHGEmissions'].describe()

In [ ]:
pd.unique(pd.concat([df_2016['OSEBuildingID'],df_2015['OSEBuildingID']])).shape

In [ ]:
len(set(pd.unique(df_2015['OSEBuildingID'])) - set(pd.unique(df_2016['OSEBuildingID'])))

Moyenne entre 2015 et 2016

In [ ]:
df_2016['mean_TotalGHGEmissions'] = df_2016['TotalGHGEmissions']
df_2016['mean_SiteEnergyUseWN(kBtu)'] = df_2016['SiteEnergyUseWN(kBtu)']

In [ ]:
for ose in df_2016['OSEBuildingID']:
    if ose in df_2015['OSEBuildingID'].values:
        row_2016 = df_2016[df_2016['OSEBuildingID'] == ose]
        ghg_2016 = row_2016['TotalGHGEmissions'].to_numpy()
        elec_2016 = row_2016['SiteEnergyUseWN(kBtu)'].to_numpy()
        row_2015 = df_2015[df_2015['OSEBuildingID'] == ose]
        ghg_2015 = row_2015['TotalGHGEmissions'].to_numpy()
        elec_2015 = row_2015['SiteEnergyUseWN(kBtu)'].to_numpy()
        df_2016.loc[df_2016['OSEBuildingID'] == ose, 'mean_TotalGHGEmissions'] = (ghg_2016+ghg_2015)/2
        df_2016.loc[df_2016['OSEBuildingID'] == ose, 'mean_SiteEnergyUseWN(kBtu)'] = (elec_2016+elec_2015)/2


In [ ]:
df = df_2016

In [ ]:
df.info()

<a name="missing-values"></a>
## Missing values

<a name="quantification"></a>
### Quantification 

In [ ]:
df.isna().sum()

In [ ]:
msno.heatmap(df)

<a name="imputation"></a>
### Imputation

In [ ]:
df_v3 = msno.nullity_filter(df, 'top', 0.5)

In [ ]:
df_v3['ENERGYSTARScore'].describe().round(2)

<a name="data-filtering"></a>
## Data filtering

<a name="row-filter"></a>
### Row filter

Drop the "multifamily" building type to only keep the non-resedential buildings.

In [ ]:
to_drop = [val for val in df_v3['BuildingType'].unique() if 'Multifamily' in val]
for val in to_drop:
    df_v3 = df_v3[df_v3['BuildingType'] != val]

Drop the negative energy building (energy production).

In [ ]:
print((df_v3["TotalGHGEmissions"]<=0).sum())

In [ ]:
print((df_v3["SiteEnergyUseWN(kBtu)"]<=0).sum())

In [ ]:
print(((df_v3["SiteEnergyUseWN(kBtu)"]<=0)|(df_v3["TotalGHGEmissions"]<=0)).sum())

In [ ]:
print((df_v3["SteamUse(kBtu)"]<0).sum())

In [ ]:
print((df_v3["Electricity(kBtu)"]<=0).sum())

In [ ]:
print(((df_v3["SiteEnergyUseWN(kBtu)"]<=0)|(df_v3["TotalGHGEmissions"]<=0)|(df_v3["Electricity(kBtu)"]<0)).sum())

In [ ]:
print((df_v3["NaturalGas(kBtu)"]<0).sum())

In [ ]:
df_v3 = df_v3[(df_v3["TotalGHGEmissions"] > 0) & (df_v3["SiteEnergyUseWN(kBtu)"]>0) & (df_v3["Electricity(kBtu)"]>0)]

<a name="column-filter"></a>
### Column filter

In [ ]:
df_v3.info()

In [ ]:
df_v3 = df_v3.drop(columns=['OSEBuildingID',
                            'DataYear', 
                            'PropertyName', 
                            'Address', 
                            'City', 
                            'State', 
                            'ZipCode',
                            'TaxParcelIdentificationNumber', 
                            'CouncilDistrictCode', 
                            'Neighborhood',
                            'ComplianceStatus',
                            'DefaultData'])

In [ ]:
print(len(df_v3['BuildingType'].unique()))
print(len(df_v3['PrimaryPropertyType'].unique()))
print(len(df_v3['ListOfAllPropertyUseTypes'].unique()))
print(len(df_v3['LargestPropertyUseType'].unique()))

<a name="distributions"></a>
## Distributions

<a name="numerical-features"></a>
### Numerical features

In [ ]:
df_num = df_v3._get_numeric_data()

In [ ]:
df_desc = df_num.describe()
df_desc.loc['var'] = df_num.var().tolist()
df_desc.loc['skew'] = df_num.skew().tolist()
df_desc.loc['kurt'] = df_num.kurtosis().tolist()
df_desc.round(2)

In [ ]:
sns.pairplot(df_v3.sample(frac=0.1), corner=True)

In [ ]:
sns.displot(x=df_v3.loc[df_v3["TotalGHGEmissions"]>0,"TotalGHGEmissions"], kind="kde", log_scale=True)

In [ ]:
sm.qqplot(df_v3["TotalGHGEmissions"].dropna(), stats.lognorm, fit=True, line="45")
plt.show()

In [ ]:
sns.displot(x=df_v3.loc[df_v3["SiteEnergyUse(kBtu)"]>0,"SiteEnergyUse(kBtu)"], kind="kde", log_scale=True)

In [ ]:
sm.qqplot(df_v3["SiteEnergyUse(kBtu)"].dropna(), stats.lognorm, fit=True, line="45")
plt.show()

<a name="categorical-features"></a>
### Categorical features

In [ ]:
df_grp = df_v3.groupby('BuildingType').size()
per_lim =0.003
df_grp = df_grp[df_grp > per_lim*len(df_v3)]
df_grp.plot(kind='pie', autopct='%.2f')

In [ ]:
df_grp = df_v3.groupby('PrimaryPropertyType').size()
per_lim =0.03
df_grp = df_grp[df_grp > per_lim*len(df_v3)]
df_grp.plot(kind='pie', autopct='%.2f')

<a name="correlations"></a>
## Correlations

In [ ]:
corr = df_v3.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df_v3.info()

In [ ]:
sns.lineplot(data=df_v3, x="YearBuilt", y="TotalGHGEmissions")

In [ ]:
e_sum = df_v3["Electricity(kBtu)"] \
        + df_v3["NaturalGas(kBtu)"] \
        + df_v3["SteamUse(kBtu)"] 
sns.scatterplot(data=df_v3, x="SiteEnergyUseWN(kBtu)", y=e_sum)

In [ ]:
res = stats.linregress(df_v3["SiteEnergyUse(kBtu)"], e_sum)
res

In [ ]:
sns.scatterplot(data=df_v3, x="TotalGHGEmissions", y=e_sum)

In [ ]:
res = stats.linregress(df_v3["TotalGHGEmissions"], e_sum)
res

In [ ]:
gfa_sum = df_v3["PropertyGFAParking"] \
        + df_v3["PropertyGFABuilding(s)"]
sns.scatterplot(data=df_v3, x="PropertyGFATotal", y=gfa_sum)

In [ ]:
res = stats.linregress(df_v3["PropertyGFATotal"], gfa_sum)
res

In [ ]:
df_v3 = df_v3.drop(columns=["PropertyGFATotal"])

In [ ]:
sns.scatterplot(data=df_v3, x="PropertyGFAParking", y="PropertyGFABuilding(s)")

In [ ]:
sns.scatterplot(data=df_v3, x="PropertyGFABuilding(s)", y="TotalGHGEmissions")

In [ ]:
sns.scatterplot(data=df_v3, x="PropertyGFABuilding(s)", y="SiteEnergyUse(kBtu)")

In [ ]:
sns.scatterplot(data=df_v3, x="PropertyGFABuilding(s)", y="mean_SiteEnergyUseWN(kBtu)")

<a name="feature-engineering"></a>
## Feature engineering

<a name="feature-engineering"></a>
### Feature engineering

In [ ]:
df_v3["is_ENERGYSTARScore"] = df_v3["ENERGYSTARScore"].isna()

In [ ]:
df_v3.groupby("is_ENERGYSTARScore")["is_ENERGYSTARScore"].count()

In [ ]:
df_v3["is_SteamUse"] = df_v3["SteamUse(kBtu)"] > 0

In [ ]:
df_v3.groupby("is_SteamUse")["is_SteamUse"].count()

In [ ]:
df_v3["is_NaturalGas"] = df_v3["NaturalGas(kBtu)"] > 0

In [ ]:
df_v3.groupby("is_NaturalGas")["is_NaturalGas"].count()

In [ ]:
df_v3["is_PropertyGFAParking"] = df_v3["PropertyGFAParking"] > 0

In [ ]:
df_v3.groupby("is_PropertyGFAParking")["is_PropertyGFAParking"].count()

In [ ]:
df_v3["ratio_SteamUse"] = (df_v3["SteamUse(kBtu)"]/(df_v3["SteamUse(kBtu)"]+df_v3["NaturalGas(kBtu)"]+df_v3["Electricity(kBtu)"])).round(1)

In [ ]:
df_v3["ratio_NaturalGas"] = (df_v3["NaturalGas(kBtu)"]/(df_v3["SteamUse(kBtu)"]+df_v3["NaturalGas(kBtu)"]+df_v3["Electricity(kBtu)"])).round(1)

In [ ]:
print(len(df_v3['BuildingType'].unique()))
print(len(df_v3['PrimaryPropertyType'].unique()))
print(len(df_v3['ListOfAllPropertyUseTypes'].unique()))
print(len(df_v3['LargestPropertyUseType'].unique()))

In [ ]:
print(df_v3['BuildingType'].unique()[:5])
print(df_v3['PrimaryPropertyType'].unique()[:5])
print(df_v3['ListOfAllPropertyUseTypes'].unique()[:5])
print(df_v3['LargestPropertyUseType'].unique()[:5])

In [ ]:
print(df_v3[['BuildingType','PrimaryPropertyType','ListOfAllPropertyUseTypes','LargestPropertyUseType']][:10])

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
embeddings = model.encode(df_v3['ListOfAllPropertyUseTypes'].tolist())
print(len(embeddings))

In [ ]:
X_embeddings = np.array(embeddings)
X_embeddings.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_components=30
pca = PCA(n_components=n_components)

In [ ]:
X_proj = pca.fit_transform(X_embeddings)

In [ ]:
print(X_proj.shape)
print(pca.explained_variance_ratio_)

In [ ]:
scree = (pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("rang de l'axe d'inertie")
plt.ylabel("pourcentage d'inertie")
plt.title("Eboulis des valeurs propres")
plt.show(block=False)

In [ ]:
def display_factorial_planes(   X_projected, 
                                x_y, 
                                pca=None, 
                                labels = None,
                                clusters=None, 
                                alpha=1,
                                figsize=[10,8], 
                                marker="." ):
    """
    Affiche la projection des individus

    Positional arguments : 
    -------------------------------------
    X_projected : np.array, pd.DataFrame, list of list : la matrice des points projetés
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2

    Optional arguments : 
    -------------------------------------
    pca : sklearn.decomposition.PCA : un objet PCA qui a été fit, cela nous permettra d'afficher la variance de chaque composante, default = None
    labels : list ou tuple : les labels des individus à projeter, default = None
    clusters : list ou tuple : la liste des clusters auquel appartient chaque individu, default = None
    alpha : float in [0,1] : paramètre de transparence, 0=100% transparent, 1=0% transparent, default = 1
    figsize : list ou tuple : couple width, height qui définit la taille de la figure en inches, default = [10,8] 
    marker : str : le type de marker utilisé pour représenter les individus, points croix etc etc, default = "."
    """

    # Transforme X_projected en np.array
    X_ = np.array(X_projected)

    # On définit la forme de la figure si elle n'a pas été donnée
    if not figsize: 
        figsize = (7,6)

    # On gère les labels
    if  labels is None : 
        labels = []
    try : 
        len(labels)
    except Exception as e : 
        raise e

    # On vérifie la variable axis 
    if not len(x_y) ==2 : 
        raise AttributeError("2 axes sont demandées")   
    if max(x_y )>= X_.shape[1] : 
        raise AttributeError("la variable axis n'est pas bonne")   

    # on définit x et y 
    x, y = x_y

    # Initialisation de la figure       
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # On vérifie s'il y a des clusters ou non
    c = None if clusters is None else clusters
 
    # Les points    
    # plt.scatter(   X_[:, x], X_[:, y], alpha=alpha, 
    #                     c=c, cmap="Set1", marker=marker)
    sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=c)

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe 
    if pca : 
        v1 = str(round(100*pca.explained_variance_ratio_[x]))  + " %"
        v2 = str(round(100*pca.explained_variance_ratio_[y]))  + " %"
    else : 
        v1=v2= ''

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f'F{x+1} {v1}')
    ax.set_ylabel(f'F{y+1} {v2}')

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() *1.1
    y_max = np.abs(X_[:, y]).max() *1.1

    # On borne x et y 
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom= -y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color='grey', alpha=0.8)
    plt.plot([0,0], [-y_max, y_max], color='grey', alpha=0.8)

    # Affichage des labels des points
    if len(labels) : 
        # j'ai copié collé la fonction sans la lire
        for i,(_x,_y) in enumerate(X_[:,[x,y]]):
            plt.text(_x, _y+0.05, labels[i], fontsize='14', ha='center',va='center') 

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})")
    plt.show()

In [ ]:
x_y = (0,1)
display_factorial_planes(X_proj, x_y, clusters=df_v3['PrimaryPropertyType'])

In [ ]:
x_y = (2,3)
display_factorial_planes(X_proj, x_y, clusters=df_v3['PrimaryPropertyType'])

In [ ]:
x_y = (4,5)
display_factorial_planes(X_proj, x_y, clusters=df_v3['PrimaryPropertyType'])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
Y_E = df_v3["SiteEnergyUseWN(kBtu)"]

In [ ]:
X_train, X_test, Y_E_train, Y_E_test = train_test_split(X_proj, Y_E, test_size=0.3, random_state=6)
print(f"[INFO] X_test shape: {X_test.shape}")
print(f"[INFO] X_train shape: {X_train.shape}")

In [ ]:
lr_E = LinearRegression()
lr_E.fit(X_train, Y_E_train)
print(f"Train score:{lr_E.score(X_train, Y_E_train)}")
print(f"Test score:{lr_E.score(X_test, Y_E_test)}")

In [ ]:
N_C = 10

In [ ]:
lr_E = LinearRegression()
lr_E.fit(X_train[:,:N_C ], Y_E_train)
print(f"Train score:{lr_E.score(X_train[:,:N_C ], Y_E_train)}")
print(f"Test score:{lr_E.score(X_test[:,:N_C ], Y_E_test)}")

In [ ]:
df_embeddings = pd.DataFrame(X_proj, columns=[f"PCA - {i}" for i in range(1,31)])

In [ ]:
df_v3 = pd.concat([df_v3, df_embeddings], axis=1)

<a name="exports"></a>
### Exports

In [ ]:
df_v3.info()

In [ ]:
df_v3.to_csv("data_cleaned_v3.csv", index=False)